# El Gamal

https://asecuritysite.com/encryption/go_el_homo

* Bob creates public key $(g,p,y = g^x \mod p)$ and private key $x$ where $p \in \mathbb{P}$.
* Alice creates msg $m$ and selects a random $k$.
* Alice computes $(a = g^k \mod p, b= y^k m \mod p)$
* Bob decrypts with $m = \frac{b}{a^x} \mod p$

In [2]:
from Crypto.PublicKey import ElGamal
from Crypto.Util import number
from ecpy.curves import Curve # https://ec-python.readthedocs.io/en/latest/

In [ ]:
key = ElGamal.generate(256, ElGamal.Random.urandom)

In [ ]:
print(f'Public key components: \n(g={key.g}, p={key.p},y={key.y})')
print(f"Private key component: \n(x={key.x})")

In [ ]:
# Alice sets parameters
g = int(key.g)
p = int(key.p)
x = int(key.x)
y = pow(g,x,p)

# Bob encrypts
m = 150
k = number.getRandomNBitInteger(128)
a = pow(g, k, p)
b = pow(y, k, p) * m % p

assert pow(pow(a, -1, p), x, p) * b % p == m

print(f"Bob's msg is (a = {a}, b = {b})")

m == pow(pow(a, x, p), -1, p) * b % p

## Multiplication with El Gamal

* Alice computes: $(a_1 = g^{k_1} \mod p, b_1= y^{k_1} m_1 \mod p)$
* Bob computes: $(a_2 = g^{k_2} \mod p, b_2= y^{k_2} m_2 \mod p)$

\begin{align}
    \frac{b_1 b_2}{a_1^x a_2^x} \mod p = \frac{(g^x)^{k_1} \cdot m_1 \cdot (g^x)^{k_2} \cdot m_2}{(g^{k_1})^x \cdot (g^{k_2})^x} \mod p = m_1 \cdot m_2 \mod p
\end{align}

## Division with El Gamal

* Alice computes: $(a_1 = g^{k_1} \mod p, b_1= y^{k_1} m_1 \mod p)$
* Bob computes: $(a_2 = g^{k_2} \mod p, b_2= y^{k_2} m_2 \mod p)$

\begin{align}
    \frac{b_1 a_2^x}{a_1^x b_2} \mod p = \frac{ (g^x)^{k_1} \cdot m_1 \cdot (g^{k_2})^x }{(g^{k_1})^x \cdot (g^x)^{k_2} \cdot m_2} \mod p = \frac{m_1}{m_2} \mod p
\end{align}

## Addition with El Gamal

* Alice computes: $(a_1 = g^{k_1} \mod p, b_1= y^{k_1} g^{m_1} \mod p)$
* Bob computes: $(a_2 = g^{k_2} \mod p, b_2= y^{k_2} g^{m_2} \mod p)$

\begin{align}
    \frac{b_1 b_2}{a_1^x a_2^x} \mod p = \frac{(g^x)^{k_1} \cdot g^{m_1} \cdot (g^x)^{k_2} \cdot g^{m_2}}{(g^{k_1})^x \cdot (g^{k_2})^x} \mod p = g^{m_1} \cdot g^{m_2} \mod p = g^{m_1 + m_2} \mod p
\end{align}

## Elliptic curve El Gamal

* Alice generates private key $a$ and public key $A = a G$ where $G$ is EC generator point
* Bob computes $M = A m$ where $m$ is his message
* Bob computes $K = k G$ where $k \leftarrow \texttt{rand()}$ and $C = kA + M$
* Alice gets $(K,M)$ and decrypts using her private key
    1. $S = aK$
    2. $M = C - S = kA + M - aK = kaG + M - akG$

**Note** Not sure what good this does as $M$ is a point multiplication $mA$ and as long as $m$ is large enough, it will be hard to find $m$ from $M$.

In [3]:
# https://github.com/dedis/kyber
# https://asecuritysite.com/encryption/go_elgamal_ecc
# The curve
curve = Curve.get_curve('Curve25519')
G = curve.generator
order = curve.order

# Alice's parts
private_key = number.getRandomNBitInteger(128)
A = private_key * G

# Bob 
M = 12345 * G
k = number.getRandomNBitInteger(128)
K = k * G
C = k * A + M

# Alice gets (K, M)
S = private_key * K
M == (C - S)

True